In [16]:
import _init_paths
from fast_rcnn.train import get_training_roidb, train_net
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from datasets.factory import get_imdb
import datasets.imdb
import caffe
import argparse
import pprint
import numpy as np
import sys
import zl_config as C
from fast_rcnn.test import im_detect
import matplotlib.pyplot as plt
from fast_rcnn.nms_wrapper import nms

CLASSES = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')

def vis_detections(im, class_name, dets, thresh=0.5):
    """Draw detected bounding boxes."""
    inds = np.where(dets[:, -1] >= thresh)[0]
    if len(inds) == 0:
        return

    im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')
    for i in inds:
        bbox = dets[i, :4]
        score = dets[i, -1]

        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='red', linewidth=3.5)
        )
        ax.text(bbox[0], bbox[1] - 2,
                '{:s} {:.3f}'.format(class_name, score),
                bbox=dict(facecolor='blue', alpha=0.5),
                fontsize=14, color='white')

    ax.set_title(('{} detections with '
                  'p({} | box) >= {:.1f}').format(class_name, class_name,
                                                  thresh),
                 fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.draw()

In [2]:
def combined_roidb(imdb_names):
    def get_roidb(imdb_name):
        imdb = get_imdb(imdb_name)
        print 'Loaded dataset `{:s}` for training'.format(imdb.name)
        imdb.set_proposal_method(cfg.TRAIN.PROPOSAL_METHOD)
        print 'Set proposal method: {:s}'.format(cfg.TRAIN.PROPOSAL_METHOD)
        roidb = get_training_roidb(imdb)
        return roidb

    roidbs = [get_roidb(s) for s in imdb_names.split('+')]
    roidb = roidbs[0]
    if len(roidbs) > 1:
        for r in roidbs[1:]:
            roidb.extend(r)
        imdb = datasets.imdb.imdb(imdb_names)
    else:
        imdb = get_imdb(imdb_names)
    return imdb, roidb


cfg_from_file('experiments/cfgs/rfcn_end2end.yml')

In [3]:
imdb, roidb = combined_roidb('voc_0712_train')

Loaded dataset `voc_0712_train` for training
Set proposal method: gt
Appending horizontally-flipped training examples...


voc_0712_train gt roidb loaded from /home/zawlin/g/iccv2017/data/cache/voc_0712_train_gt_roidb.pkl
done
Preparing training data...


I0112 15:47:52.070415 21966 roidb.py:31] loading roidb from /media/zawlin/ssd/iccv2017/_cache/voc_0712_train.roidb.pkl


done


In [4]:
import cv2

ann = roidb[4000]
im = cv2.imread(ann['image'])
idx = 0
for bb in ann['boxes']:   
    cv2.rectangle(im,(bb[0],bb[1]),(bb[2],bb[3]),(0,255,0),1)
cv2.imshow('im2',im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
prototxt = 'models/pascal_voc/ResNet-101/rfcn_end2end/test_agnostic.prototxt'
model = 'data/rfcn_models/resnet101_rfcn_final.caffemodel'
caffe.set_mode_gpu()
caffe.set_device(0)
net = caffe.Net(prototxt, model, caffe.TEST)

In [8]:
im = cv2.imread('data/demo/004545.jpg')
scores, boxes = im_detect(net, im)

In [11]:
print boxes.shape

(300, 8)


In [22]:
CONF_THRESH = 0.8
NMS_THRESH = 0.3
im_render = im.copy()
for cls_ind, cls in enumerate(CLASSES[1:]):
    cls_ind += 1 #because we skipped background
    cls_boxes = boxes[:, 4:8]
    cls_scores = scores[:, cls_ind]
    dets = np.hstack((cls_boxes,
                      cls_scores[:, np.newaxis])).astype(np.float32)
    keep = nms(dets, NMS_THRESH)
    dets = dets[keep, :]
    inds = np.where(dets[:, -1] >= CONF_THRESH)[0]
    if len(inds)>0:
        for d in dets[inds]:
            cv2.rectangle(im_render,(d[0],d[1]),(d[2],d[3]),(255,0,0),1)
cv2.imshow('imrender',im_render)
cv2.waitKey(0)
cv2.destroyAllWindows()